In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
gln_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Gln_FA_13.xlsx'
df_gln1 = load_data(gln_path)
gln_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/14_Gln_A.xlsx'
df_gln1_alkb = load_data(gln_path)

gln_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_B_Gln_FA_19.xlsx'
df_gln2 = load_data(gln_path)
gln_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_B_Gln_A_FA_20.xlsx'
df_gln2_alkb = load_data(gln_path)

In [ ]:
df_gln1 = df_gln1[df_gln1.Mass>1500]
df_gln1_alkb = df_gln1_alkb[df_gln1_alkb.Mass>1500]
df_gln2 = df_gln2[df_gln2.Mass>1500]
df_gln2_alkb = df_gln2_alkb[df_gln2_alkb.Mass>1500]

In [ ]:
plotly_zone(df_gln2[df_gln2.Mass>23900], y='Vol')

In [ ]:
plotly_zone(df_gln2)
df_gln2.shape

In [ ]:
_, df_alkb1_only = diff_dfs(df_gln1, df_gln1_alkb)
df_alkb1_only.shape

In [ ]:
_, df_alkb2_only = diff_dfs(df_gln2, df_gln2_alkb)
df_alkb2_only.shape

In [ ]:
# remove adducts from df_gln2
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_gln2, df_gln2, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_gln2_without_adducts = df_gln2.drop(dfm_adducts.index)

In [ ]:
# remove adducts from df_alkb2_only
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_alkb2_only, df_alkb2_only, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_alkb2_only_without_adducts = df_alkb2_only.drop(dfm_adducts.index)

In [ ]:
# remove adducts from df_gln2
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_gln2_alkb, df_gln2_alkb, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_gln2_alkb_without_adducts = df_gln2_alkb.drop(dfm_adducts.index)

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
# params = [(df_intact_wild2, df_intact_alkb2_only, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_gln1, df_alkb1_only, shift) for shift in np.arange(-64, 64, 1)]
params = [(df_gln2_without_adducts, df_gln2_alkb_without_adducts, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_comm_wild2, df_comm_wild2, shift) for shift in np.arange(1, 64, 1)]
# params = [(df_wild2, df_alkb2, shift) for shift in np.arange(1, 100, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [ ]:
fig = px.scatter(df_shift_hits, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
df_shift_hits.sort_values('hit', ascending=False).reset_index(drop=True).iloc[:10].copy()

In [ ]:
# df_sh_1 = df_shift_hits.copy()
df_sh_2 = df_shift_hits.copy()

In [ ]:
shift = -1*M
dfm_l, dfm_r = peer_dfs(df_gln2_without_adducts, df_gln2_alkb_without_adducts, shift=shift)
dfm_l.shape, dfm_r.shape

In [ ]:
# plotly_zone(dfm_r[(dfm_r.Mass>5800)&(dfm_r.Mass<6800)], y='Vol')
# plotly_zone(dfm_l[(dfm_l.Mass>5800)&(dfm_l.Mass<6800)], y='Vol')
plotly_zone(dfm_l, y='RT')
plotly_zone(dfm_r, y='RT')

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_sample, shift):
    dfm = match_dfs(df_sample, df_sample, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
params = [(df_alkb_comm_N_inc, shift) for shift in np.arange(1, 64, 1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [ ]:
fig = px.scatter(df_shift_hits, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
bcr = base_calling_random(dfm_l, base_only=True)
plotly_basecalling(*bcr)

In [ ]:
bcr = base_calling_random(dfm_r, base_only=True)
plotly_basecalling(*bcr)

In [ ]:
thres = 26000
groups_l = basecalling_groups(dfm_l[dfm_l.Mass<thres])
len(groups_l)
groups_r = basecalling_groups(dfm_r[dfm_r.Mass<thres])
len(groups_r)

In [ ]:
print([len(g[0]) for g in groups_l]), print([len(g[0]) for g in groups_r])

In [ ]:
# [len(group[0]) for group in groups]
idx = 0

dft = groups_l[idx][0]
bcr = base_calling_random(dft)
plotly_basecalling(*bcr)

dft = groups_r[idx][0]
bcr = base_calling_random(dft)
plotly_basecalling(*bcr)

In [ ]:
shift = -2*M
dfm_l, dfm_r = peer_dfs(df_gln2_without_adducts, df_gln2_without_adducts, shift=shift)
dfm_l.shape, dfm_r.shape

In [ ]:
bcr = base_calling_random(dfm_l[dfm_l.Mass<6000], base_only=True)
plotly_basecalling(*bcr)
bcr = base_calling_random(dfm_r[dfm_r.Mass<6000], base_only=True)
plotly_basecalling(*bcr)